In [1]:
#Loading packages
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
import re
#import scrape_functions_un as kzd
import scraping_model as sm 

## Players Dictionary

In [ ]:
players_names = {'Rory McIlroy': '17419',
            'Tiger Woods': '10085', 
            'Jordan Spieth': '19976',
            'Justin Thomas': '20037',
            'Scottie Scheffler': '25786',
            'Collin Morikawa': '33992',
            'Sergio García': '10170',
            'Dustin Johnson': '17976',
            'Phil Mickelson': '10083',
            'Ian Poulter': '11956',
            'Talor Gooch': '27168',
            'Cameron Smith': '27755',
            'Joaquín Niemann': '33997',
            'Bryson DeChambeau': '29148',
            'Brooks Koepka': '25730',
            'Jon Rahm': '28177'}

### We want to get all the dates and urls for each of the player's pages

In [2]:
niemann_url = 'https://www.asapsports.com/show_player.php?id=33997'

response = requests.get(niemann_url)
soup = BeautifulSoup(response.text, 'html.parser')
dates_niemann = soup.find_all('b')

In [3]:
## Retrieving player interview links
links = []
for row in dates_niemann:
        a_tags = soup.find_all('a', href=True)
        
for atag in a_tags:
    url = atag['href']
    if url.startswith('http://www.asapsports.com/show_interview.php?id='):
         links.append(url)

In [ ]:
links[0:5]

In [ ]:
nie_df = pd.DataFrame({'Links' :links})   

In [ ]:
nie_df

In [4]:

URL = 'http://www.asapsports.com/show_interview.php?id=194496'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Getting Data
name = soup.find('h1')
dates = soup.find('h2')
event = soup.find('h3')

print(name.text)
print(dates.text)
print(event.text)


LIV GOLF JEDDAH
March 3, 2024
Joaquin Niemann


In [5]:
interviews = []
dates, answers = sm.parse_interview(URL)
niemann_result = pd.DataFrame({'date':dates,'answer':answers})

In [6]:
niemann_result

,date,answer
0,"March 3, 2024","(joaquin niemann, [yeah, it, feels, amazing, i..."
1,"March 3, 2024","(joaquín niemann, [i, wouldn't, say, it, felt,..."
2,"March 3, 2024","(joaquin niemann, [i, mean, at, least, that's,..."
3,"March 3, 2024","(joaquin niemann, [i, believe, so, yeah, i, fe..."
4,"March 3, 2024","(joaquin niemann, [oh, that, was, beautiful, y..."
5,"March 3, 2024","(joaquin niemann, [yeah, i, used, to, feel, a,..."
6,"March 3, 2024","(joaquin niemann, [it's, been, streaky, i've, ..."
7,"March 3, 2024","(joaquin niemann, [yeah, obviously, there, is,..."
8,"March 3, 2024","(joaquin niemann, [yeah, that, was, my, goal, ..."
9,"March 3, 2024","(joaquin niemann, [yeah, gary's, role, was, hu..."


In [7]:
#for i,n in niemann_result['answer']:
    #print(i)

niemann_result['player'] = [i for i, n in niemann_result['answer']]
niemann_result['answer'] = [n for i, n in niemann_result['answer']]
niemann_result

,date,answer,player
0,"March 3, 2024","[yeah, it, feels, amazing, it's, a, super, spe...",joaquin niemann
1,"March 3, 2024","[i, wouldn't, say, it, felt, that, way, at, th...",joaquín niemann
2,"March 3, 2024","[i, mean, at, least, that's, what, i, think, i...",joaquin niemann
3,"March 3, 2024","[i, believe, so, yeah, i, feel, like, it's, pr...",joaquin niemann
4,"March 3, 2024","[oh, that, was, beautiful, yeah, that, hole, i...",joaquin niemann
5,"March 3, 2024","[yeah, i, used, to, feel, a, lot, more, my, ri...",joaquin niemann
6,"March 3, 2024","[it's, been, streaky, i've, been, having, good...",joaquin niemann
7,"March 3, 2024","[yeah, obviously, there, is, a, lot, more, i'l...",joaquin niemann
8,"March 3, 2024","[yeah, that, was, my, goal, today, was, to, tr...",joaquin niemann
9,"March 3, 2024","[yeah, gary's, role, was, huge, not, only, tod...",joaquin niemann


In [8]:
def get_questions(bold_tags):
    questions = []
    for tag in bold_tags:
        if tag.text.startswith('Q.'):
            parent_p = tag.find_parent('p')
            if parent_p:
                question_text = parent_p.get_text(strip=True)
                questions.append(question_text)
    return questions

In [10]:
bold_text = soup.find_all('b')
questions = get_questions(bold_text) 
stripped_questions = [question.strip('Q.') for question in questions]
stripped_questions = [sm.clean_string(question).split() for question in stripped_questions]

# Si no hay pregunta para la respuesta, quiero que rellenes con 'MODERATOR' 

since the moderator questions don't have a fixed pattern, we struggle to retrieve them, we will impute them with 'moderator' in the question; in the text analysis we might get rid of the token so it doesn't consider it an important token.

In [11]:
num_to_fill = len(niemann_result['answer']) - len(stripped_questions)
stripped_questions = ["MODERATOR"] * num_to_fill + stripped_questions

In [12]:
niemann_result['question'] = stripped_questions

In [13]:
niemann_result['Ans LIV?'] =niemann_result['answer'].apply(lambda x: 1 if 'liv' in x else 0)
niemann_result['Ans PGA?'] =niemann_result['answer'].apply(lambda x: 1 if 'pga' in x else 0)
niemann_result['Q LIV?'] =niemann_result['question'].apply(lambda x: 1 if 'liv' in x else 0)
niemann_result['Q PGA?'] =niemann_result['question'].apply(lambda x: 1 if 'pga' in x else 0)


In [14]:
niemann_result

,date,answer,player,question,Ans LIV?,Ans PGA?,Q LIV?,Q PGA?
0,"March 3, 2024","[yeah, it, feels, amazing, it's, a, super, spe...",joaquin niemann,MODERATOR,0,0,0,0
1,"March 3, 2024","[i, wouldn't, say, it, felt, that, way, at, th...",joaquín niemann,"[you, looked, in, control, out, there, all, da...",0,0,0,0
2,"March 3, 2024","[i, mean, at, least, that's, what, i, think, i...",joaquin niemann,"[the, way, you're, playing, right, now, do, yo...",0,0,0,0
3,"March 3, 2024","[i, believe, so, yeah, i, feel, like, it's, pr...",joaquin niemann,"[is, this, as, good, of, a, confidence, level,...",0,0,0,0
4,"March 3, 2024","[oh, that, was, beautiful, yeah, that, hole, i...",joaquin niemann,"[there, was, one, shot, i, think, it, was, the...",0,0,0,0
5,"March 3, 2024","[yeah, i, used, to, feel, a, lot, more, my, ri...",joaquin niemann,"[you, talked, yesterday, about, an, adjustment...",0,0,0,0
6,"March 3, 2024","[it's, been, streaky, i've, been, having, good...",joaquin niemann,"[you, feel, like, you're, a, better, putter, now]",0,0,0,0
7,"March 3, 2024","[yeah, obviously, there, is, a, lot, more, i'l...",joaquin niemann,"[that, draw, is, working, so, nicely, at, the,...",0,0,0,0
8,"March 3, 2024","[yeah, that, was, my, goal, today, was, to, tr...",joaquin niemann,"[what, was, the, best, thing, about, today, yo...",0,0,0,0
9,"March 3, 2024","[yeah, gary's, role, was, huge, not, only, tod...",joaquin niemann,"[we, saw, on, the, live, feed, that, you, were...",0,0,0,0


In [ ]:
URL_base = 'http://www.asapsports.com/show_interview.php?id='
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Getting Data
name = soup.find('h1')
dates = soup.find('h2')
event = soup.find('h3')

print(name.text)
print(dates.text)
print(event.text)


links = []
for row in dates_niemann:
        a_tags = soup.find_all('a', href=True)
        
for atag in a_tags:
    url = atag['href']
    if url.startswith('http://www.asapsports.com/show_interview.php?id='):
         links.append(url)

### For loop for all the links in the players page

In [19]:
press_conferences = pd.DataFrame(columns=['date','answer','player','question'])
names = []
dates = []
event = []
for link in links:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Getting Data
    name = soup.find('h1').text
    date = soup.find('h2').text
    event = soup.find('h3').text

    # names.append(name)
    # dates.append(date)
    # event.append(event)

    print(name,'\t', date,'\t', event)
    # dates, answers = sm.parse_interview(URL)
    
    # press_conferences['player'] = [i for i, n in press_conferences['answer']]
    # press_conferences['answer'] = [n for i, n in press_conferences['answer']]
    # bold_text = soup.find_all('b')
    # questions = get_questions(bold_text) 
    # stripped_questions = [question.strip('Q.') for question in questions]
    # stripped_questions = [sm.clean_string(question).split() for question in stripped_questions]

    # num_to_fill = len(press_conferences['answer']) - len(stripped_questions)
    # stripped_questions = ["MODERATOR"] * num_to_fill + stripped_questions
    # stripped_questions['question'] = stripped_questions

    # stripped_questions['Ans LIV?'] =stripped_questions['answer'].apply(lambda x: 1 if 'liv' in x else 0)
    # stripped_questions['Ans PGA?'] =stripped_questions['answer'].apply(lambda x: 1 if 'pga' in x else 0)
    # stripped_questions['Q LIV?'] =stripped_questions['question'].apply(lambda x: 1 if 'liv' in x else 0)
    # stripped_questions['Q PGA?'] =stripped_questions['question'].apply(lambda x: 1 if 'pga' in x else 0)



LIV GOLF JEDDAH 	 March 3, 2024 	 Joaquin Niemann
LIV GOLF JEDDAH 	 March 2, 2024 	 Joaquin Niemann
LIV GOLF JEDDAH 	 March 1, 2024 	 Jon Rahm
LIV GOLF MAYAKOBA 	 February 4, 2024 	 Joaquin Niemann
LIV GOLF MAYAKOBA 	 February 3, 2024 	 Joaquin Niemann
LIV GOLF MAYAKOBA 	 February 2, 2024 	 Joaquin Niemann
LIV GOLF MAYAKOBA 	 February 1, 2024 	 Joaquin Niemann
LIV GOLF TEAM CHAMPIONSHIP MIAMI 	 October 21, 2023 	 Joaquin Niemann
LIV GOLF TEAM CHAMPIONSHIP MIAMI 	 October 20, 2023 	 Dustin Johnson
LIV GOLF TEAM CHAMPIONSHIP MIAMI 	 October 19, 2023 	 Bubba Watson
LIV GOLF GREENBRIER 	 August 6, 2023 	 Joaquin Niemann
LIV GOLF GREENBRIER 	 August 4, 2023 	 Joaquin Niemann
LIV GOLF GREENBRIER 	 August 2, 2023 	 Joaquin Niemann
LIV GOLF ANDALUCIA 	 July 2, 2023 	 Joaquin Niemann
LIV GOLF ANDALUCIA 	 June 29, 2023 	 Joaquin Niemann
LIV GOLF WASHINGTON DC 	 May 28, 2023 	 Joaquin Niemann
MASTERS TOURNAMENT 	 April 9, 2023 	 Joaquin Niemann
MASTERS TOURNAMENT 	 April 7, 2023 	 Joaquin Niemann

## TENEMOS UN PROBLEMA, HAY CONFERENCIAS QUE TIENEN MAS DE UN JUGADOR RESPONDIENDO. ESTÁN INTERCALADAS LAS PREGUNTAS ENTONCES TENEMOS QUE ASIGNAR EL JUGADOR DEPENDIENDO DE LA RTA Y NO DE LA CONFERENCIA EN GRAL.

## HAY QUE COJER LAS FECHAS PARA LUEGO FILTRAR A PARTIR DE OCTUBRE DE 2020